<div align="center">

# <span style="font-size: 2em;">Welcome to Linear Regression!</span>

</div>

<p align="center">
  <img src="img_perceptron-Photoroom.png" alt="Header" />
</p>

<div align="center">

# <span style="font-size: em;">Supervised Learning!</span>

</div>